# Converter for the Financial Filings at the Sec

In [189]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2010q1.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t")
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2010q1"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

1365228
883780
17820
100377
24168


In [190]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------------------------------------------+--------------+--------------------------------------------+----------+--------+-------+-------------+----------------------------+
|    | adsh                 | tag                                         | version      | coreg                                      |    ddate |   qtrs | uom   |       value | footnote                   |
|----+----------------------+---------------------------------------------+--------------+--------------------------------------------+----------+--------+-------+-------------+----------------------------|
|  0 | 0001104659-10-010697 | AccountsAndNotesReceivableNet               | us-gaap/2009 | nan                                        | 20090131 |      0 | USD   | 3.4754e+09  | nan                        |
|  1 | 0001104659-10-010697 | AccountsAndNotesReceivableNet               | us-gaap/2009 | nan                                        | 20091031 |      0 | USD   | 2.6169e+

In [191]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+----------+--------+--------+---------+---------+---------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                               | version      | plabel                                                                                                                               |   negating |
|----+----------------------+----------+--------+--------+---------+---------+---------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------+------------|
|  0 | 0000950123-10-018432 |        1 |     16 | BS     |       0 | X       | AccountsAndNotesReceivableNet          

In [192]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------+--------------------------------------------+-------+-------------+----------+----------------------+------------+------------------------------------------+--------------------------------+--------------------+-------------+----------+--------------------+------------+------------------------------------------+--------------------------------+--------------+-----------+-----------+--------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+-------------------+---------+--------------------------------------------------------------------------------------------------------------------------+
|    | adsh                 |     cik | name                                       |   sic | countryba   | stprba   | cityba               | zipba      | bas1                                     | bas2                           | baph               |

In [193]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))

+----+-------------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [194]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+


In [195]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                 0001104659-10-010697
tag         AccountsAndNotesReceivableNet
version                      us-gaap/2009
coreg                                 NaN
ddate                            20090131
qtrs                                    0
uom                                   USD
value                        3475400000.0
footnote                              NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [196]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [197]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [207]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 288 Dtos size: 161 Bs: 50 Cf: 111 Ic: 0 Json size: 32kb.
file 1 of 496 exportFiles/2010q1/0001193125-10-072854.json stored in 720.692ms.
File exportFiles/2010q1/0001193125-10-072909.json has Symbol Series([], )
File exportFiles/2010q1/0001193125-10-073247.json has Symbol Series([], )
FilterDfNum size: 312 Dtos size: 139 Bs: 52 Cf: 87 Ic: 0 Json size: 26kb.
file 4 of 496 exportFiles/2010q1/0000060667-10-000064.json stored in 825.357ms.
FilterDfNum size: 263 Dtos size: 167 Bs: 66 Cf: 101 Ic: 0 Json size: 35kb.
file 5 of 496 exportFiles/2010q1/0000950123-10-029721.json stored in 708.402ms.
FilterDfNum size: 270 Dtos size: 177 Bs: 90 Cf: 87 Ic: 0 Json size: 32kb.
file 6 of 496 exportFiles/2010q1/0000950123-10-029809.json stored in 730.962ms.
FilterDfNum size: 249 Dtos size: 152 Bs: 72 Cf: 80 Ic: 0 Json size: 30kb.
file 7 of 496 exportFiles/2010q1/0000950123-10-029845.json stored in 675.84ms.
File exportFiles/2010q1/0000950123-10-030079.json has Symbol Series([], )
FilterD

FilterDfNum size: 402 Dtos size: 264 Bs: 100 Cf: 164 Ic: 0 Json size: 54kb.
file 66 of 496 exportFiles/2010q1/0000950123-10-018464.json stored in 1049.518ms.
FilterDfNum size: 320 Dtos size: 177 Bs: 76 Cf: 101 Ic: 0 Json size: 33kb.
file 67 of 496 exportFiles/2010q1/0000950123-10-018519.json stored in 832.699ms.
File exportFiles/2010q1/0000950123-10-018524.json has Symbol Series([], )
FilterDfNum size: 354 Dtos size: 196 Bs: 76 Cf: 120 Ic: 0 Json size: 37kb.
file 69 of 496 exportFiles/2010q1/0000950123-10-018671.json stored in 965.571ms.
FilterDfNum size: 294 Dtos size: 206 Bs: 89 Cf: 117 Ic: 0 Json size: 40kb.
file 70 of 496 exportFiles/2010q1/0000950123-10-018679.json stored in 817.63ms.
FilterDfNum size: 471 Dtos size: 178 Bs: 16 Cf: 162 Ic: 0 Json size: 35kb.
file 71 of 496 exportFiles/2010q1/0000950123-10-018789.json stored in 1287.918ms.
FilterDfNum size: 313 Dtos size: 180 Bs: 66 Cf: 114 Ic: 0 Json size: 35kb.
file 72 of 496 exportFiles/2010q1/0000950123-10-019013.json stored in

FilterDfNum size: 355 Dtos size: 214 Bs: 76 Cf: 138 Ic: 0 Json size: 42kb.
file 130 of 496 exportFiles/2010q1/0000950123-10-017267.json stored in 977.933ms.
File exportFiles/2010q1/0000950123-10-017377.json has Symbol Series([], )
FilterDfNum size: 285 Dtos size: 177 Bs: 78 Cf: 99 Ic: 0 Json size: 33kb.
file 132 of 496 exportFiles/2010q1/0000950123-10-017583.json stored in 808.439ms.
FilterDfNum size: 316 Dtos size: 183 Bs: 84 Cf: 99 Ic: 0 Json size: 34kb.
file 133 of 496 exportFiles/2010q1/0000950123-10-017631.json stored in 876.259ms.
FilterDfNum size: 267 Dtos size: 156 Bs: 66 Cf: 90 Ic: 0 Json size: 28kb.
file 134 of 496 exportFiles/2010q1/0000950123-10-017647.json stored in 750.163ms.
FilterDfNum size: 264 Dtos size: 164 Bs: 74 Cf: 90 Ic: 0 Json size: 34kb.
file 135 of 496 exportFiles/2010q1/0000950123-10-017728.json stored in 734.805ms.
File exportFiles/2010q1/0000950123-10-017776.json has Symbol Series([], )
FilterDfNum size: 280 Dtos size: 195 Bs: 78 Cf: 117 Ic: 0 Json size: 40

FilterDfNum size: 378 Dtos size: 252 Bs: 90 Cf: 162 Ic: 0 Json size: 49kb.
file 194 of 496 exportFiles/2010q1/0001193125-10-042685.json stored in 1035.243ms.
FilterDfNum size: 326 Dtos size: 176 Bs: 80 Cf: 96 Ic: 0 Json size: 34kb.
file 195 of 496 exportFiles/2010q1/0001193125-10-042716.json stored in 897.411ms.
FilterDfNum size: 599 Dtos size: 312 Bs: 128 Cf: 184 Ic: 0 Json size: 63kb.
file 196 of 496 exportFiles/2010q1/0001193125-10-042763.json stored in 1649.245ms.
FilterDfNum size: 258 Dtos size: 177 Bs: 84 Cf: 93 Ic: 0 Json size: 37kb.
file 197 of 496 exportFiles/2010q1/0001193125-10-042765.json stored in 673.229ms.
FilterDfNum size: 269 Dtos size: 164 Bs: 74 Cf: 90 Ic: 0 Json size: 31kb.
file 198 of 496 exportFiles/2010q1/0001193125-10-042802.json stored in 699.622ms.
FilterDfNum size: 361 Dtos size: 197 Bs: 86 Cf: 111 Ic: 0 Json size: 37kb.
file 199 of 496 exportFiles/2010q1/0001193125-10-042810.json stored in 939.679ms.
FilterDfNum size: 448 Dtos size: 258 Bs: 114 Cf: 144 Ic: 0

FilterDfNum size: 380 Dtos size: 208 Bs: 94 Cf: 114 Ic: 0 Json size: 39kb.
file 254 of 496 exportFiles/2010q1/0001140361-10-008295.json stored in 1047.411ms.
FilterDfNum size: 199 Dtos size: 149 Bs: 68 Cf: 81 Ic: 0 Json size: 28kb.
file 255 of 496 exportFiles/2010q1/0001140361-10-008522.json stored in 557.543ms.
FilterDfNum size: 258 Dtos size: 122 Bs: 57 Cf: 65 Ic: 0 Json size: 23kb.
file 256 of 496 exportFiles/2010q1/0001157523-10-001218.json stored in 721.371ms.
File exportFiles/2010q1/0001193125-10-039749.json has Symbol Series([], )
File exportFiles/2010q1/0001193125-10-039864.json has Symbol Series([], )
FilterDfNum size: 264 Dtos size: 163 Bs: 70 Cf: 93 Ic: 0 Json size: 30kb.
file 259 of 496 exportFiles/2010q1/0001193125-10-040106.json stored in 734.666ms.
FilterDfNum size: 488 Dtos size: 342 Bs: 84 Cf: 258 Ic: 0 Json size: 70kb.
file 260 of 496 exportFiles/2010q1/0001193125-10-040142.json stored in 1354.414ms.
File exportFiles/2010q1/0001193125-10-040148.json has Symbol Series(

FilterDfNum size: 383 Dtos size: 209 Bs: 116 Cf: 93 Ic: 0 Json size: 38kb.
file 317 of 496 exportFiles/2010q1/0000950123-10-015829.json stored in 1109.443ms.
FilterDfNum size: 360 Dtos size: 187 Bs: 90 Cf: 97 Ic: 0 Json size: 39kb.
file 318 of 496 exportFiles/2010q1/0001047469-10-001151.json stored in 1010.629ms.
FilterDfNum size: 260 Dtos size: 165 Bs: 76 Cf: 89 Ic: 0 Json size: 30kb.
file 319 of 496 exportFiles/2010q1/0001104659-10-008799.json stored in 723.029ms.
FilterDfNum size: 348 Dtos size: 188 Bs: 82 Cf: 106 Ic: 0 Json size: 35kb.
file 320 of 496 exportFiles/2010q1/0001104659-10-008891.json stored in 989.872ms.
File exportFiles/2010q1/0001140361-10-007829.json has Symbol Series([], )
FilterDfNum size: 277 Dtos size: 166 Bs: 82 Cf: 84 Ic: 0 Json size: 35kb.
file 322 of 496 exportFiles/2010q1/0001140361-10-007923.json stored in 794.427ms.
File exportFiles/2010q1/0001193125-10-036866.json has Symbol Series([], )
FilterDfNum size: 317 Dtos size: 159 Bs: 72 Cf: 87 Ic: 0 Json size: 

FilterDfNum size: 363 Dtos size: 216 Bs: 78 Cf: 138 Ic: 0 Json size: 41kb.
file 377 of 496 exportFiles/2010q1/0001193125-10-034308.json stored in 1027.428ms.
File exportFiles/2010q1/0001193125-10-034314.json has Symbol Series([], )
FilterDfNum size: 269 Dtos size: 148 Bs: 70 Cf: 78 Ic: 0 Json size: 27kb.
file 379 of 496 exportFiles/2010q1/0000045012-10-000085.json stored in 739.913ms.
FilterDfNum size: 307 Dtos size: 177 Bs: 84 Cf: 93 Ic: 0 Json size: 34kb.
file 380 of 496 exportFiles/2010q1/0000067716-10-000030.json stored in 861.207ms.
FilterDfNum size: 206 Dtos size: 127 Bs: 58 Cf: 69 Ic: 0 Json size: 24kb.
file 381 of 496 exportFiles/2010q1/0000702165-10-000049.json stored in 551.13ms.
FilterDfNum size: 276 Dtos size: 140 Bs: 56 Cf: 84 Ic: 0 Json size: 30kb.
file 382 of 496 exportFiles/2010q1/0000930413-10-000862.json stored in 773.033ms.
File exportFiles/2010q1/0000950123-10-013710.json has Symbol Series([], )
FilterDfNum size: 216 Dtos size: 152 Bs: 104 Cf: 48 Ic: 0 Json size: 33

FilterDfNum size: 230 Dtos size: 138 Bs: 54 Cf: 84 Ic: 0 Json size: 26kb.
file 443 of 496 exportFiles/2010q1/0001193125-10-023294.json stored in 649.036ms.
FilterDfNum size: 239 Dtos size: 130 Bs: 66 Cf: 64 Ic: 0 Json size: 26kb.
file 444 of 496 exportFiles/2010q1/0001193125-10-023332.json stored in 680.551ms.
FilterDfNum size: 179 Dtos size: 106 Bs: 64 Cf: 42 Ic: 0 Json size: 19kb.
file 445 of 496 exportFiles/2010q1/0001206774-10-000241.json stored in 504.349ms.
File exportFiles/2010q1/0001004155-10-000016.json has Symbol Series([], )
File exportFiles/2010q1/0001193125-10-020962.json has Symbol Series([], )
File exportFiles/2010q1/0001193125-10-021555.json has Symbol Series([], )
FilterDfNum size: 201 Dtos size: 150 Bs: 78 Cf: 72 Ic: 0 Json size: 28kb.
file 449 of 496 exportFiles/2010q1/0001193125-10-021684.json stored in 561.836ms.
FilterDfNum size: 255 Dtos size: 136 Bs: 68 Cf: 68 Ic: 0 Json size: 27kb.
file 450 of 496 exportFiles/2010q1/0001193125-10-021715.json stored in 719.519ms